In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn import datasets
import scipy.signal
import scipy.cluster.vq as vq
import scipy.io.wavfile as wav
import python_speech_features as speech
from sklearn import mixture
from sklearn.mixture import GaussianMixture as gmm
import scipy.spatial.distance as dist

In [31]:
def BIC(k, Koefs, Covs, N, penalty):
    return 0.5*N[Koefs[k][0]]* np.log(np.linalg.det(Covs[Koefs[k][0]])) + 0.5*N[Koefs[k][1]] * np.log(np.linalg.det(Covs[Koefs[k][1]]))-0.5* (N[Koefs[k][0]]+N[Koefs[k][1]])*np.log(np.linalg.det(Covs[Koefs[k][0]][:3600, :3600], Covs[Koefs[k][1]][:3600, :3600])) + penalty(2.5*(N[Koefs[k][0]]+N[Koefs[k][1]]))

In [32]:
def fit_samples(samples):
	gmix = mixture.GaussianMixture(n_components=5, covariance_type='full')
	gmix.fit(samples)
	return (gmix.means_) #ovo su centroide

#glavna funkcija:
def calculate(Sigs, num, penalty):
    mfcc_feat = np.copy(speech.mfcc(Sigs[0]))
    Means = np.zeros((1, num))
    N = np.zeros((1, num))#mfcc lengths
    Covs = []
    Freqs = []
    
    for i in range(0, num):
        mfcc = speech.mfcc(Sigs[i])
        if(i>0):
            mfcc_feat = np.vstack((mfcc_feat, speech.mfcc(Sigs[i])))
        Means[0][i] = np.mean(mfcc)
        N[0][i] = mfcc.shape[0]
        Covs.append(np.cov(mfcc))

    gmms = fit_samples(mfcc_feat)
    vq.whiten(mfcc_feat)
    kmeans = vq.kmeans(obs=mfcc_feat, k_or_guess=gmms)
    
    for i in range(0, num):
        freq = kmeans[0][i]
        freq_sum = np.sum(freq)
        freq = (freq_sum - freq)/freq_sum
        Freqs.append(freq)
    
    M = np.zeros((num,num))
    for i in range(0,num):
        for j in range(0,num):
            if(i==j):
                M[i][j]=0
            else:
                M[i][j] = np.abs(dist.cosine(Freqs[i], Freqs[j]))
                
    Koefs = []
    for i in range(0, num):
        Koefs.append([0,0])
    MaxVals = np.zeros((num))
    
    #racunanje N najblizih koeficijenata
    for i in range(0,num-1):
        for j in range(2,num):
            if(j>i):
                
                for i in range(0,num):
                    if(M[i][j]>=MaxVals[i]):
                        for k in range(i+1, num):
                            MaxVals[k]=MaxVals[k-1]
                            Koefs[k] = Koefs[k-1]
                        MaxVals[i] = M[i][j]
                        Koefs[i] = [i,j]
                        
    #racunanje najveceg BIC
    Bics = []
    for i in range(0, num):
        Bics.append(0)
        
    for i in range(0, num):
        Bics[i] = BIC(i, Koefs, Covs, N, penalty)
    
    max_bic = Bics[0]
    for i in range(1,num):
        if(Bics[i]>max_bic):
            max_bic = Bics[i]
            max_i = i
            
    if(max_bic<0):
        return (Sigs, -1)
    
    [i,j] = Koefs[max_i]
    sig_new = 0.5*Sigs[i]+0.5*Sigs[j]
    Sigs_new = []
    
    for k in range (0,num):
        if(k!=i and k!=j):
            Sigs_new.append(Sigs[k])
            
    return (Sigs_new, 1)

In [30]:
(rate,sig1) = wav.read("richard3.wav")
(rate,sig2) = wav.read("amy3.wav")
(rate,sig3) = wav.read("derek3.wav")
(rate,sig4) = wav.read("paolo3.wav")
(rate,sig5) = wav.read("nilofer3.wav")

Sigs =[sig1, sig2, sig3, sig4, sig5]

ind = 1
num = 2
while(ind>0):
    (Sigs, ind) = calculate(Sigs, num, ind)
    num = num-1

Sigs

C:\Users\Zoran\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
C:\Users\Zoran\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: invalid value encountered in subtract
  from ipykernel import kernelapp as app


TypeError: 'int' object is not callable